In [ ]:
%pip install feedparser pymongo redis pandas

In [ ]:
import hashlib
import feedparser
import redis
import pandas as pd

In [ ]:
rss_feeds = {
    'example': {'link': 'https://www.example.com/rss/'},
}

In [ ]:
def checkHubBub(url):
    feed = feedparser.parse(url)

    # Look for 'links' in the feed's 'feed' attribute
    if 'links' in feed.feed:
        for link in feed.feed.links:
            if link['rel'] == 'hub':
                return "pubsubhubbub" in link['href']

In [ ]:
for feedName, data in rss_feeds.items():
  print(feedName)
  data["hasHubbub"] = checkHubBub(data["link"])
  data["entries"] = feedparser.parse(data["link"]).entries



In [ ]:
records = []

for feedName, data in rss_feeds.items():
  for entry in data["entries"]:
    id = entry.get('guid', '')
    title = entry.get('title', '')
    link = entry.get('link', '')
    tags = tags = [tag.term for tag in entry.get('tags', [])]

    content = entry.get('summary')
    if not content or len(content) < 10:
      content_data = entry.get('content', [{}])
      if isinstance(content_data, list) and isinstance(content_data[0], dict):
        content = content_data[0].get('value', '')

    if not content or len(content) < 10:
      content = title

    records.append({
        'source': feedName,
        'id': id,
        'title': title,
        'link': link,
        'content': content,
        'tags': tags
    })

In [ ]:
df = pd.DataFrame(records)

In [ ]:
df.head()